# 第 6 章 数据加载、存储与文件格式

介绍 pandas 的数据输入与输出

## 6.1 读写文本格式的数据

我将大致介绍一下这些函数在将文本数据转换为 DataFrame 时所用到的一些技术。

In [2]:
import pandas as pd
import numpy as np

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
df = pd.read_csv('examples/ex1.csv')

In [6]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


我们还可以使用 read_table，并指定分隔符

In [7]:
pd.read_table('examples/ex1.csv', sep=',')

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

读入该文件的办法有两个。你可以让 pandas 为其分配默认的列名，也可以自己 定义列名:

In [12]:
#方法1:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
#方法2:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd',
'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


假设你希望将 message 列做成 DataFrame 的索引。你可以明确表示要将该列放 到索引 4 的位置上，也可以通过 index_col 参数指定"message":

In [14]:
names = ['a', 'b', 'c', 'd', 'message']

In [15]:
pd.read_csv('examples/ex2.csv', names=names,
index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表
即可:

In [16]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [17]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])

In [18]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [19]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [20]:
# 正则表达式表达为\s+    (\s 匹配任意的空白符 ,+ 重复一次或更多次)
result = pd.read_table('examples/ex3.txt', sep='\s+')

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [21]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


你可以用 skiprows 跳过文件的第一行、第三行和第 四行:

In [22]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [23]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有 (空字符串)，要么用某个标记值表示。默认情况下，pandas 会用一组经常出 现的标记值进行识别，比如 NA 及 NULL:

In [24]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [25]:
result = pd.read_csv('examples/ex5.csv')

In [26]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values 可以用一个列表或集合的字符串表示缺失值:

In [28]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])

In [30]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


字典的各列可以使用不同的 NA 标记值:

In [31]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [32]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

In [33]:
pd.options.display.max_rows = 10

In [34]:
result = pd.read_csv('examples/ex6.csv')

In [35]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


如果只想读取几行(避免读取整个文件)，通过 nrows 进行指定即可:

In [36]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


要逐块读取文件，可以指定 chunksize(行数):

In [38]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [39]:
chunker

我们可以迭代处理 ex6.csv，将值计数聚合到"key"列中， 如下所示:

In [40]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [41]:
tot = pd.Series([])

In [43]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [44]:
tot = tot.sort_values(ascending=False)

In [45]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextParser 还有一个 get_chunk 方法，它使你可以读取任意大小的块。

## 将数据写出到文本格式

数据也可以被输出为分隔符格式的文本。我们再来看看之前读过的一个 CSV 文 件:

In [50]:
data = pd.read_csv('examples/ex5.csv')

In [51]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


利用 DataFrame 的 to_csv 方法，我们可以将数据写到一个以逗号分隔的文件 中:

In [52]:
data.to_csv('examples/out.csv')

In [53]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


当然，还可以使用其他分隔符(由于这里直接写出到 sys.stdout，所以仅仅是 打印出文本结果而已):

In [54]:
import sys

In [55]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记
值

In [56]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


如果没有设置其他选项，则会写出行和列的标签。当然，它们也都可以被禁用:

In [64]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


你还可以只写出一部分的列，并以你指定的顺序排列:

In [62]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b',
'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series 也有一个 to_csv 方法:

In [67]:
dates = pd.date_range('1/1/2000', periods=7)

In [69]:
ts = pd.Series(np.arange(7), index=dates)

In [70]:
ts.to_csv('examples/tseries.csv')

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [71]:
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 处理分隔符格式

In [72]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


对于任何单字符分隔符文件，可以直接使用 Python 内置的 csv 模块。将任意已 打开的文件或文件型的对象传给 csv.reader:

In [74]:
import csv
f = open('examples/ex7.csv')

In [75]:
reader = csv.reader(f)

对这个 reader 进行迭代将会为每行产生一个元组(并移除了所有的引号):

In [76]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


首先，读取文件到一个多行的列表中:

In [77]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

我们将这些行分为标题行和数据行:

In [78]:
header, values = lines[0], lines[1:]

我们可以用字典构造式和 zip(*values)，后者将行转置为列，创建数据 列的字典:

In [79]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [80]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV 文件的形式有很多。只需定义 csv.Dialect 的一个子类即可定义出新格式 (如专门的分隔符、字符串引用约定、行结束符等):

class my_dialect(csv.Dialect): lineterminator = '\n' delimiter = ';' quotechar = '"' quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)

reader = csv.reader(f, delimiter='|')

with open('mydata.csv', 'w') as f: writer = csv.writer(f, dialect=my_dialect) writer.writerow(('one', 'two', 'three')) writer.writerow(('1', '2', '3')) writer.writerow(('4', '5', '6')) writer.writerow(('7', '8', '9'))


## JSON 数据

JSON(JavaScript Object Notation 的简称)已经成为通过 HTTP 请求在 Web 浏览器和其他应用程序之间发送数据的标准格式之一。它是一种比表格型文本 格式(如 CSV)灵活得多的数据格式。

In [84]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [85]:
import json

In [86]:
result = json.loads(obj)

In [87]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps 则将 Python 对象转换成 JSON 格式:

In [89]:
asjson = json.dumps(result)

In [90]:
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

最简单方便的方式是:向 DataFrame 构造器传入一个字典的列 表(就是原先的 JSON 对象)，并选取数据字段的子集:

In [91]:
siblings = pd.DataFrame(result['siblings'], columns=['name',
'age'])

In [92]:
siblings

,name,age
0,Scott,30
1,Katie,38


pandas.read_json 可以自动将特别格式的 JSON 数据集转换为 Series 或 DataFrame。

In [93]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


pandas.read_json 的默认选项假设 JSON 数组中的每个对象是表格中的一行:

In [95]:
data = pd.read_json('examples/example.json')

In [96]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


如果你需要将数据从 pandas 输出到 JSON，可以使用 to_json 方法:

In [97]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [98]:
print(data.to_json(orient='records'))    #指示预期的JSON字符串格式。

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML 和 HTML:Web 信息收集

Python 有许多可以读写常见的 HTML 和 XML 格式数据的库，包括 lxml、 Beautiful Soup 和 html5lib。

pandas 有一个内置的功能，read_html，它可以使用 lxml 和 Beautiful Soup 自动将 HTML 文件中的表格解析为 DataFrame 对象。

你需要安装 read_html 用到的库:

pip install lxml
pip install beautifulsoup4 html5lib

pandas.read_html 有一些选项，默认条件下，它会搜索、尝试解析<table>标 签内的的表格数据。结果是一个列表的 DataFrame 对象:

In [101]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [102]:
len(tables)

1

In [103]:
failures = tables[0]

In [104]:
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [105]:
failures.head()  #首5行

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


计算
按年份计算倒闭的银行数:


In [106]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [107]:
close_timestamps.dt.year.value_counts()  #按照年来统计


2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

## 利用 lxml.objectify 解析 XML  (没看懂)

XML(Extensible Markup Language)是另一种常见的支持分层、嵌套数据以及 元数据的结构化数据格式。

我们先用 lxml.objectify 解析该文件，然后通过 getroot 得到该 XML 文件的根 节点的引用:

In [108]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [113]:
parsed

In [114]:
root

<Element a at 0x11996d5c8>

In [109]:

data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [119]:
data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'PERIOD_YEAR': 2008,
  'PERIOD_MONTH': 1,
  'CATEGORY': 'Service Indicators',
  'FREQUENCY': 'M',
  'INDICATOR_UNIT': '%',
  'YTD_TARGET': 95.0,
  'YTD_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'MONTHLY_ACTUAL': 96.9},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate 

In [110]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [111]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [116]:
root

<Element a at 0x11996d5c8>

In [117]:
root.get('href')

'http://www.google.com'

In [118]:
root.text

'Google'

# 6.2 二进制数据格式

实现数据的高效二进制格式存储最简单的办法之一是使用 Python 内置的 pickle 序列化。

pandas 对象都有一个用于将数据以 pickle 格式保存到磁盘上 的 to_pickle 方法:

In [120]:
frame = pd.read_csv('examples/ex1.csv')

In [121]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [123]:
frame.to_pickle('examples/frame_pickle')    #以 pickle 格式保存到磁盘上 的 to_pickle

In [124]:
pd.read_pickle('examples/frame_pickle')     #你可以通过 pickle 直接读取被 pickle 化的数据，或是使用更为方便的 pandas.read_pickle

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pandas 内置支持两个二进制数据格式:HDF5 和 MessagePack。

## 使用 HDF5 格式

HDF5 中的 HDF 指的 是层次型数据格式(hierarchical data format)。·

虽然可以用 PyTables 或 h5py 库直接访问 HDF5 文件，pandas 提供了更为高级 的接口，可以简化存储 Series 和 DataFrame 对象。HDFStore 类可以像字典一 样，处理低级的细节:

In [3]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [10]:
frame

,a
0,-0.295726
1,-0.219549
2,0.914041
3,-1.151589
4,1.475524
5,1.756269
6,0.256289
7,-0.868250
8,-0.202643
9,-1.353687


In [4]:
store = pd.HDFStore('mydata.h5')

In [5]:
store['obj1'] = frame

In [8]:
store['obj1_col'] = frame['a']

In [9]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [11]:
store['obj1']

,a
0,-0.295726
1,-0.219549
2,0.914041
3,-1.151589
4,1.475524
5,1.756269
6,0.256289
7,-0.868250
8,-0.202643
9,-1.353687


HDFStore 支持两种存储模式，'fixed'和'table'。后者通常会更慢，但是支持 使用特殊语法进行查询操作:

In [13]:
store.put('obj2', frame, format='table')

In [14]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.247264
11,0.527590
12,-0.204456
13,-0.182710
14,0.445818
15,0.353000


In [15]:
store.close()

put 是 store['obj2'] = frame 方法的显示版本，允许我们设置其它的选项， 比如格式。

pandas.read_hdf 函数可以快捷使用这些工具:

In [17]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [18]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.295726
1,-0.219549
2,0.914041
3,-1.151589
4,1.475524


In [ ]:
os.remove('mydata.h5')

## 读取 Microsoft Excel 文件

pandas 的 ExcelFile 类或 pandas.read_excel 函数支持读取存储在 Excel 2003 (或更高版本)中的表格型数据。这两个工具分别使用扩展包 xlrd 和 openpyxl 读取 XLS 和 XLSX 文件。

要使用 ExcelFile，通过传递 xls 或 xlsx 路径创建一个实例:

In [19]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [20]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [21]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要将 pandas 数据写入为 Excel 格式，你必须首先创建一个 ExcelWriter， 然后使用 pandas 对象的 to_excel 方法将数据写入到其中:

In [22]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [23]:
frame.to_excel(writer, 'Sheet1')

In [24]:
writer.save()

你还可以不使用 ExcelWriter，而是传递文件的路径到 to_excel:

In [30]:
frame.to_excel('examples/ex2.xlsx')

In [28]:
!rm examples/ex2.xlsx

# 6.3 Web APIs 交互

我们可以发一个 HTTP GET 请 求，使用 requests 扩展库:

In [33]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [34]:

data = resp.json()
data[0]['title']

'BUG: Ensure that fill_na in Categorical only replaces null values'

In [35]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,27932,BUG: Ensure that fill_na in Categorical only r...,[],open
1,27931,xtick label have wrong position.,[],open
2,27929,[DEPR] Deprecate numeric_only parameter in Cat...,"[{'id': 78527356, 'node_id': 'MDU6TGFiZWw3ODUy...",open
3,27928,Assignment of column via .loc for numpy non-ns...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
4,27927,BUG: DataFrame._get_value not working for Inte...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
5,27926,Fix regression in .ix fallback with IntervalIndex,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
6,27925,BUG: Help python csv engine read binary buffers,"[{'id': 47229171, 'node_id': 'MDU6TGFiZWw0NzIy...",open
7,27922,"CLN: Remove incorrect check, comment, rename","[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
8,27921,BUG: Lambda function returns KeyError in DataF...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
9,27920,Updated table with read_fwf(),"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open


# 6.4 数据库交互

我将使用 SQLite 数据库(通过 Python 内置的 sqlite3 驱动器):

In [36]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [38]:
#插入数据
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [39]:
# 从表中选取数据时，大部分 Python SQL 驱动器(PyODBC、psycopg2、 MySQLdb、pymssql 等)都会返回一个元组列表:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

你可以将这个元组列表传给 DataFrame 构造器，但还需要列名(位于光标的 description 属性中):

In [41]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [40]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


SQLAlchemy 项目是一个流行的 Python SQL 工具，它抽象出了 SQL 数据库中的 许多常见差异。pandas 有一个 read_sql 函数，可以让你轻松的从 SQLAlchemy 连接读取数据。这里，我们用 SQLAlchemy 连接 SQLite 数据库，并从之前创建 的表读取数据:

In [42]:

import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


In [45]:
!rm mydata.sqlite

# 6.5 总结